# SGS Analytical Workflow Notebook

This notebook walks through core analytical concepts for geologic and geospatial data. It is designed to run on real datasets if you provide file paths, or generate synthetic data when paths are left empty.


## Setup and User Data Paths

Provide file paths to your data files (GeoTIFF or GeoJSON). Leave any path empty to use synthetic data.


In [ ]:
# =============================================================================
# USER CONFIGURATION
# =============================================================================
DATA_CONFIG = {
    # Rasters
    "continuous_raster_path": None,  # GeoTIFF with continuous values
    "categorical_raster_path": None,  # GeoTIFF with class labels

    # Vectors
    "vector_path": None,  # GeoJSON or GeoPackage
    "geochem_points_path": None,  # GeoJSON with geochemistry points

    # Spectral halo classification
    "spectral_indices_dir": None,  # Folder of spectral index GeoTIFFs

    # Prospectivity mapping
    "prospectivity_feature_rasters": [],  # List of raster paths (GeoTIFF)
    "prospectivity_training_points_path": None,  # GeoJSON with known deposits
}


## Imports


In [ ]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import geopandas as gpd
import rasterio
from rasterio.transform import from_bounds

from scipy.ndimage import gaussian_filter
from scipy.spatial import KDTree
from scipy.spatial.distance import cdist

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')


## Load Helpers and Prepare Data


In [ ]:
import sys
sys.path.append(str(Path.cwd()))
import helpers as h


def load_raster(path):
    with rasterio.open(path) as src:
        data = src.read(1)
        extent = (src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top)
        return data, extent, src.crs


def load_vector(path):
    gdf = gpd.read_file(path)
    return gdf


def ensure_xy(gdf):
    if 'X' not in gdf.columns or 'Y' not in gdf.columns:
        gdf = gdf.copy()
        gdf['X'] = gdf.geometry.x
        gdf['Y'] = gdf.geometry.y
    return gdf


# --- Continuous raster ---
if DATA_CONFIG['continuous_raster_path']:
    continuous_raster, raster_extent, raster_crs = load_raster(DATA_CONFIG['continuous_raster_path'])
else:
    continuous_raster, raster_extent = h.generate_synthetic_raster()
    raster_crs = 'EPSG:32610'

# --- Categorical raster ---
if DATA_CONFIG['categorical_raster_path']:
    categorical_raster, _, _ = load_raster(DATA_CONFIG['categorical_raster_path'])
else:
    # Simple synthetic categorical raster derived from continuous
    quantiles = np.quantile(continuous_raster[~np.isnan(continuous_raster)], [0.25, 0.5, 0.75])
    categorical_raster = np.digitize(continuous_raster, bins=quantiles)

# --- Vector data ---
if DATA_CONFIG['vector_path']:
    vector_gdf = load_vector(DATA_CONFIG['vector_path'])
else:
    gdf_points, gdf_lines, gdf_polygons = h.generate_synthetic_vector_geometries()
    vector_gdf = gdf_points

# --- Geochemistry points ---
if DATA_CONFIG['geochem_points_path']:
    geochem_gdf = load_vector(DATA_CONFIG['geochem_points_path'])
else:
    geochem_gdf = h.generate_synthetic_geochemistry()

geochem_gdf = ensure_xy(geochem_gdf)

print('Raster shape:', continuous_raster.shape)
print('Vector records:', len(vector_gdf))
print('Geochem records:', len(geochem_gdf))


## Problem Formulation

Define the analytical goal and decision context up front. Examples include:
- Predicting prospectivity from multi-source geoscience data.
- Detecting multivariate geochemical anomalies.
- Interpolating a sparse sample grid to continuous surfaces.

Clarify what constitutes a positive target (e.g., mineralization, alteration halo), what output format is required (map, ranked targets, clusters), and which constraints apply (data quality, spatial resolution, computational cost).


## Data Format Considerations


In [ ]:
# Raster vs vector comparison
h.plot_raster_vs_vector(continuous_raster, geochem_gdf, extent=raster_extent)
plt.show()

# Point vs line vs polygon (synthetic)
if 'gdf_points' not in globals():
    gdf_points, gdf_lines, gdf_polygons = h.generate_synthetic_vector_geometries()

h.plot_geometry_types(gdf_points, gdf_lines, gdf_polygons)
plt.show()

# Continuous vs categorical rasters
h.plot_continuous_vs_categorical(continuous_raster, categorical_raster, extent=raster_extent)
plt.show()


## EDA, Transformations, and Scaling


In [ ]:
# Identify numeric feature columns
numeric_cols = geochem_gdf.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [c for c in numeric_cols if c not in ['X', 'Y']]
if not feature_cols:
    raise ValueError('No numeric feature columns found in geochem data.')

value_col = feature_cols[0]
values = geochem_gdf[value_col].values

# Distribution and log transform
h.plot_distribution(values, title=f'Distribution: {value_col}')
plt.show()

log_values = np.log1p(values - np.nanmin(values) + 1)
h.plot_transformation_comparison(values, log_values, transform_name='Log1p')
plt.show()

# Scaling
scaler = StandardScaler()
scaled_features = scaler.fit_transform(geochem_gdf[feature_cols])
scaled_df = pd.DataFrame(scaled_features, columns=feature_cols)

h.plot_correlation_matrix(scaled_df, title='Scaled Feature Correlations')
plt.show()


## Missing Data and Imputation


In [ ]:
# Inject missing values for demonstration
geochem_missing = h.add_missing_data(geochem_gdf, missing_pct=0.1, columns=feature_cols)

h.plot_missing_data_pattern(geochem_missing[feature_cols])
plt.show()

# Mean imputation
imputer = SimpleImputer(strategy='mean')
imputed_values = imputer.fit_transform(geochem_missing[feature_cols])

h.plot_imputation_comparison(
    geochem_missing[value_col].values,
    imputed_values[:, feature_cols.index(value_col)],
    column_name=value_col
)
plt.show()


## Bias and Data Leakage (Spatial Autocorrelation)

Geologic data are often spatially autocorrelated, which inflates model performance when random train/test splits are used. Use spatially aware validation (e.g., block cross-validation or buffered splits) to reduce leakage.


In [ ]:
values = geochem_gdf[value_col].values
h.plot_spatial_autocorrelation(geochem_gdf, values, title='Spatial Autocorrelation')
plt.show()


## Analytical Methods


### Interpolation (IDW + Kriging)


In [ ]:
# Prepare sample points and grid
sample_coords = geochem_gdf[['X', 'Y']].values
sample_values = geochem_gdf[value_col].values

xmin, xmax, ymin, ymax = raster_extent
grid_resolution = 60
x_grid = np.linspace(xmin, xmax, grid_resolution)
y_grid = np.linspace(ymin, ymax, grid_resolution)
xx, yy = np.meshgrid(x_grid, y_grid)
grid_points = np.column_stack([xx.ravel(), yy.ravel()])

# IDW interpolation

def idw_interpolation(sample_coords, sample_values, grid_points, power=2, n_neighbors=12):
    tree = KDTree(sample_coords)
    distances, indices = tree.query(grid_points, k=n_neighbors)
    distances = np.maximum(distances, 1e-10)
    weights = 1 / (distances ** power)
    weights_sum = weights.sum(axis=1, keepdims=True)
    weights_normalized = weights / weights_sum
    interpolated = np.sum(weights_normalized * sample_values[indices], axis=1)
    return interpolated

idw_pred = idw_interpolation(sample_coords, sample_values, grid_points, power=2)
idw_grid = idw_pred.reshape(grid_resolution, grid_resolution)

h.plot_interpolation_results(sample_coords, sample_values, idw_grid, raster_extent, method_name='IDW')
plt.show()

# Empirical semivariogram

def compute_semivariogram(coords, values, n_lags=12, max_lag=None):
    dist_matrix = cdist(coords, coords)
    if max_lag is None:
        max_lag = np.percentile(dist_matrix, 50)

    lag_edges = np.linspace(0, max_lag, n_lags + 1)
    lag_centers = (lag_edges[:-1] + lag_edges[1:]) / 2

    semivariance = []
    for i in range(n_lags):
        mask = (dist_matrix > lag_edges[i]) & (dist_matrix <= lag_edges[i + 1])
        if mask.sum() > 0:
            ii, jj = np.where(mask)
            sq_diff = (values[ii] - values[jj]) ** 2
            semivariance.append(0.5 * np.mean(sq_diff))
        else:
            semivariance.append(np.nan)

    return lag_centers, np.array(semivariance)

lags, semivar = compute_semivariogram(sample_coords, sample_values)

# Spherical variogram model

def spherical_variogram(h, nugget, sill, range_param):
    h = np.asarray(h)
    result = np.zeros_like(h, dtype=float)

    mask = h > 0
    h_norm = h[mask] / range_param

    within_range = h_norm <= 1
    result_temp = np.zeros_like(h_norm)
    result_temp[within_range] = nugget + (sill - nugget) * (
        1.5 * h_norm[within_range] - 0.5 * h_norm[within_range] ** 3
    )
    result_temp[~within_range] = sill

    result[mask] = result_temp
    return result


def fit_variogram(lags, semivar):
    from scipy.optimize import curve_fit

    valid = ~np.isnan(semivar)
    lags_clean = lags[valid]
    semivar_clean = semivar[valid]

    if len(lags_clean) < 3:
        return 0, np.max(semivar_clean), np.max(lags_clean)

    nugget_init = semivar_clean[0] if semivar_clean[0] > 0 else 0
    sill_init = np.max(semivar_clean)
    range_init = np.max(lags_clean) / 2

    try:
        popt, _ = curve_fit(
            spherical_variogram,
            lags_clean,
            semivar_clean,
            p0=[nugget_init, sill_init, range_init],
            bounds=([0, 0, 1], [sill_init, sill_init * 2, np.max(lags_clean) * 2]),
            maxfev=5000,
        )
        return popt
    except Exception:
        return nugget_init, sill_init, range_init


def ordinary_kriging(sample_coords, sample_values, grid_points, nugget, sill, range_param, n_neighbors=12):
    tree = KDTree(sample_coords)
    predictions = np.zeros(len(grid_points))
    variances = np.zeros(len(grid_points))

    for i, point in enumerate(grid_points):
        distances, indices = tree.query(point, k=n_neighbors)

        if np.min(distances) < 1e-10:
            predictions[i] = sample_values[indices[0]]
            variances[i] = 0
            continue

        local_coords = sample_coords[indices]
        local_values = sample_values[indices]

        n = len(local_coords)
        K = np.zeros((n + 1, n + 1))

        for j in range(n):
            for k in range(n):
                dist = np.linalg.norm(local_coords[j] - local_coords[k])
                K[j, k] = sill - spherical_variogram(dist, nugget, sill, range_param)

        K[n, :n] = 1
        K[:n, n] = 1
        K[n, n] = 0

        k0 = np.zeros(n + 1)
        for j in range(n):
            dist = np.linalg.norm(local_coords[j] - point)
            k0[j] = sill - spherical_variogram(dist, nugget, sill, range_param)
        k0[n] = 1

        try:
            weights = np.linalg.solve(K, k0)
            predictions[i] = np.dot(weights[:n], local_values)
            variances[i] = sill - np.dot(weights[:n], k0[:n]) - weights[n]
        except Exception:
            w = 1 / (distances ** 2)
            predictions[i] = np.sum(w * local_values) / np.sum(w)
            variances[i] = np.var(local_values)

    return predictions, np.maximum(variances, 0)

nugget, sill, range_param = fit_variogram(lags, semivar)
model_fit = spherical_variogram(lags, nugget, sill, range_param)

h.plot_semivariogram(lags, semivar, model_fit=model_fit, title='Fitted Variogram')
plt.show()

kriging_pred, kriging_var = ordinary_kriging(
    sample_coords, sample_values, grid_points, nugget, sill, range_param, n_neighbors=12
)

kriging_grid = kriging_pred.reshape(grid_resolution, grid_resolution)
variance_grid = kriging_var.reshape(grid_resolution, grid_resolution)

h.plot_interpolation_results(sample_coords, sample_values, kriging_grid, raster_extent, method_name='Ordinary Kriging')
plt.show()

h.plot_raster(variance_grid, title='Kriging Variance', extent=raster_extent, cmap='YlOrRd')
plt.show()


### PCA


In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(geochem_gdf[feature_cols])

pca = PCA(n_components=min(5, len(feature_cols)))
_ = pca.fit_transform(scaled_features)

h.plot_pca_results(pca, feature_cols)
plt.show()


### K-means (Geochemical Populations)


In [ ]:
k_range = range(2, 8)
inertias = []
silhouettes = []

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(scaled_features)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(scaled_features, labels))

h.plot_elbow_silhouette(list(k_range), inertias, silhouettes)
plt.show()

best_k = list(k_range)[int(np.argmax(silhouettes))]
km = KMeans(n_clusters=best_k, random_state=42, n_init=10)
labels = km.fit_predict(scaled_features)

pca_2 = PCA(n_components=2)
pca_2_features = pca_2.fit_transform(scaled_features)

centers_2d = pca_2.transform(km.cluster_centers_)

h.plot_clustering_results(pca_2_features, labels, centers=centers_2d,
                          title=f'K-means Clusters (k={best_k})', feature_x=0, feature_y=1)
plt.show()


### Multivariate Anomaly Detection (Isolation Forest)

See full reference implementation in `anomaly_detection/`.


In [ ]:
iso = IsolationForest(n_estimators=200, contamination=0.05, random_state=42)
iso_labels = iso.fit_predict(scaled_features)
scores = -iso.decision_function(scaled_features)

h.plot_anomaly_scores(geochem_gdf, scores, binary_labels=iso_labels, title='Isolation Forest')
plt.show()


### Spectral Halo Classification

See full reference implementation in `MinersWork/spectral_unsupervised/spectral_unsupervised_class.ipynb`.


In [ ]:
# Load spectral indices from disk if provided, otherwise create synthetic indices
spectral_indices = {}

if DATA_CONFIG['spectral_indices_dir']:
    dir_path = Path(DATA_CONFIG['spectral_indices_dir'])
    spectral_extent = None
    for tif_path in dir_path.glob('*.tif'):
        data, extent, _ = load_raster(tif_path)
        spectral_indices[tif_path.stem] = data
        if spectral_extent is None:
            spectral_extent = extent
    if spectral_extent is None:
        spectral_extent = raster_extent
else:
    base = continuous_raster
    spectral_indices = {
        'NDVI': base + np.random.normal(0, 1.0, base.shape),
        'NDMI': gaussian_filter(base, sigma=3),
        'BSI': np.flipud(base) + np.random.normal(0, 1.0, base.shape),
    }
    spectral_extent = raster_extent

# Compute smoothed presence maps
presence_maps = []
for name, arr in spectral_indices.items():
    threshold = np.nanquantile(arr, 0.9)
    presence = (arr >= threshold).astype(float)
    smoothed = gaussian_filter(presence, sigma=5)
    presence_maps.append(smoothed)

stack = np.stack(presence_maps, axis=-1)
stack_2d = stack.reshape(-1, stack.shape[-1])

# K-means to classify halo types
n_classes = 5
km = KMeans(n_clusters=n_classes, random_state=42, n_init=10)
labels = km.fit_predict(stack_2d)
class_map = labels.reshape(stack.shape[0], stack.shape[1])

h.plot_alteration_map(class_map)
plt.show()


### Supervised ML Prospectivity Mapping

See full reference implementation in `supervised_ML/`.


In [ ]:
# Synthetic raster stack (or load from user paths)
if DATA_CONFIG['prospectivity_feature_rasters']:
    feature_stack = []
    for path in DATA_CONFIG['prospectivity_feature_rasters']:
        data, extent, _ = load_raster(path)
        feature_stack.append(data)
    prospect_extent = extent
    feature_stack = np.stack(feature_stack, axis=-1)
else:
    r1 = continuous_raster
    r2 = gaussian_filter(continuous_raster, sigma=4)
    r3 = np.flipud(continuous_raster)
    feature_stack = np.stack([r1, r2, r3], axis=-1)
    prospect_extent = raster_extent

rows, cols, n_features = feature_stack.shape

# Training data
if DATA_CONFIG['prospectivity_training_points_path']:
    train_gdf = load_vector(DATA_CONFIG['prospectivity_training_points_path'])
    label_candidates = [c for c in train_gdf.columns if c.lower() in ['label', 'target', 'prospectivity']]
    if not label_candidates:
        raise ValueError('Training points file must include a label column like label/target/prospectivity.')
    label_col = label_candidates[0]

    train_gdf = ensure_xy(train_gdf)
    xmin, xmax, ymin, ymax = prospect_extent

    col_idx = ((train_gdf['X'] - xmin) / (xmax - xmin) * (cols - 1)).astype(int)
    row_idx = ((ymax - train_gdf['Y']) / (ymax - ymin) * (rows - 1)).astype(int)
    valid = (col_idx >= 0) & (col_idx < cols) & (row_idx >= 0) & (row_idx < rows)

    X = feature_stack[row_idx[valid], col_idx[valid], :]
    y = train_gdf.loc[valid, label_col].values
else:
    # Synthetic labels from a high-value anomaly
    label_raster = (feature_stack[..., 0] > np.nanpercentile(feature_stack[..., 0], 92)).astype(int)

    n_samples = 2000
    rng = np.random.default_rng(42)
    row_idx = rng.integers(0, rows, n_samples)
    col_idx = rng.integers(0, cols, n_samples)

    X = feature_stack[row_idx, col_idx, :]
    y = label_raster[row_idx, col_idx]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

# Predict prospectivity across the grid
flat_features = feature_stack.reshape(-1, n_features)
prob = rf.predict_proba(flat_features)[:, 1].reshape(rows, cols)

h.plot_prospectivity_map(prob, extent=prospect_extent, threshold=0.6)
plt.show()

# Evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test)))
